In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from itertools import islice
import gc
from natsort import natsorted

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint

images_path_base = '/home/bernhard/Documents/ml/freesound/tex_down/'
images_ch = images_path_base + 'ch_tex'
images_melsp = images_path_base + 'melsp_tex'
images_mfcc = images_path_base + 'mfcc_tex'
images_sh = images_path_base + 'sh_tex'
images_sp = images_path_base + 'sp_tex'

m = len(os.listdir(images_ch))

cols = ['fname', 'label', 'manually_verified']
df = pd.read_csv('train_post_competition.csv', usecols=cols)

# Kick out all non-manually verified files
df = df[df['manually_verified'] == 1]
df = df.drop('manually_verified', axis=1)

X_files = np.asarray(df)
Y = X_files[:m,1]
X_files = []

def load_images(path):
  image_list = natsorted(os.listdir(path))
  loaded_images = []
  for image in islice(image_list, m):
    with open(os.path.join(path, image), 'rb') as i:
      img = Image.open(i)
      data = np.asarray(img, dtype='int32')
      # data = data[:,:,:3]
      loaded_images.append(data)
  loaded_images = np.array(loaded_images)
  return loaded_images

X_ch = load_images(images_ch)
X_melsp = load_images(images_melsp)
X_mfcc = load_images(images_mfcc)
X_sh = load_images(images_sh)
X_sp = load_images(images_sp)

def reshape_tex(X):
  return X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))

X_ch = reshape_tex(X_ch)
X_melsp = reshape_tex(X_melsp)
X_mfcc = reshape_tex(X_mfcc)
X_sh = reshape_tex(X_sh)
X_sp = reshape_tex(X_sp)

print('X shape: {}'.format(X_ch.shape))
print('Y shape: {}'.format(Y.shape))
print('Number tex: {}'.format(m))

Y_one_hot = OneHotEncoder(sparse=False)
Y = Y.reshape(Y.shape[0], 1)
Y_one_hot_tex = Y_one_hot.fit_transform(Y)
print('Number of categories: {}'.format(len(Y_one_hot_tex[0])))

random_seed = 2
_, X_ch_val, _, _ = train_test_split(X_ch, Y_one_hot_tex,
                     test_size = 0.2, random_state=random_seed)

_, X_melsp_val, _, Y_val = train_test_split(X_melsp, Y_one_hot_tex,
                     test_size = 0.2, random_state=random_seed)

_, X_mfcc_val, _, _ = train_test_split(X_mfcc, Y_one_hot_tex,
                     test_size = 0.2, random_state=random_seed)

_, X_sh_val, _, _ = train_test_split(X_sh, Y_one_hot_tex,
                     test_size = 0.2, random_state=random_seed)

_, X_sp_val, _, _ = train_test_split(X_sp, Y_one_hot_tex,
                     test_size = 0.2, random_state=random_seed)

# Clear some memory
del df
X = []
gc.collect()

model_mfcc = Sequential([
  Conv2D(32, (3,3), strides=1, input_shape=(X_ch_val.shape[1:]), activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(64, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(128, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(256, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dropout(0.25),
  Dense(41, activation='softmax'),
])

model_sh = Sequential([
  Conv2D(32, (3,3), strides=1, input_shape=(X_ch_val.shape[1:]), activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(64, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(128, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(256, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dropout(0.25),
  Dense(41, activation='softmax'),
])

model_sp = Sequential([
  Conv2D(32, (3,3), strides=1, input_shape=(X_ch_val.shape[1:]), activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(64, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(128, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(256, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dropout(0.25),
  Dense(41, activation='softmax'),
])

model_melsp = Sequential([
  Conv2D(32, (3,3), strides=1, input_shape=(X_ch_val.shape[1:]), activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(64, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(128, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(256, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Dropout(0.25),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
  Dropout(0.25),
  Dense(41, activation='softmax'),
])

model_ch = Sequential([
  Conv2D(32, (3,3), strides=1, input_shape=(X_ch_val.shape[1:]), activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(64, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(128, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
  Conv2D(256, (3,3), strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2,2)),
#   Dropout(0.25),
  Flatten(),
  Dense(256, activation='relu'),
  Dense(128, activation='relu'),
  Dense(64, activation='relu'),
#   Dropout(0.25),
  Dense(41, activation='softmax'),
])

model_ch.load_weights('weights/model_ch.h5')
model_melsp.load_weights('weights/model_melsp.h5')
model_mfcc.load_weights('weights/model_mfcc.h5')
model_sh.load_weights('weights/model_sh.h5')
model_sp.load_weights('weights/model_sp.h5')

X shape: (3704, 100, 135, 1)
Y shape: (3704,)
Number tex: 3704
Number of categories: 41


In [45]:
preds_ch = model_ch.predict(X_ch_val, verbose=0)
preds_melsp = model_melsp.predict(X_melsp_val, verbose=0)
preds_mfcc = model_mfcc.predict(X_mfcc_val, verbose=0)
preds_sh = model_sh.predict(X_sh_val, verbose=0)
preds_sp = model_sp.predict(X_sp_val, verbose=0)

preds_total = (preds_ch + preds_melsp + preds_mfcc + preds_sh + preds_sp) / 5
print(preds_total.shape)

(741, 41)


In [47]:
preds_total_one_hot = [[i == max(arr) for i in arr] for arr in preds_total]

preds = Y_one_hot.inverse_transform(preds_total_one_hot)
Y_val_orig = Y_one_hot.inverse_transform(Y_val)

comparison = [i == j for i, j in zip(preds, Y_val_orig)]
print(np.mean(comparison))

0.8218623481781376


In [44]:
# ch, melsp, mfcc, sh, sp
preds_one_hot = [[i == max(arr) for i in arr] for arr in preds_mfcc]
preds = Y_one_hot.inverse_transform(preds_one_hot)
comparison = [i == j for i, j in zip(preds, Y_val_orig)]
print(np.mean(comparison))

0.6207827260458839
